# Agente LLM Personalizado
---

Este cuaderno explica cómo crear su propio agente LLM personalizado.

Un agente LLM consta de tres partes:

- `PromptTempalte`: Esta es la plantilla de indicaciones que se puede utilizar para indicarle al modelo de lenguaje qué hacer
- `LLM`: Este es el modelo de lenguaje que impulsa al agente.
- `stop sequence`: Le indica al LLM que deje de generar tan pronto como se encuentre esta cadena
- `OutputParser`: Determina cómo analizar la salida de LLM en un objeto AgentAction o AgentFinish.

El Agente LLM se utiliza en un `AgentExecutor`. Este `AgentExecutor` puede considerarse en gran medida como un bucle que:

1. Pasa la entrada del usuario y cualquier paso previo al Agente (en este caso, el Agente LLM)
2. Si el Agente devuelve un `AgentFinish`, luego devuélvalo directamente al usuario.
3. Si el Agente devuelve una `AgentAction`, úsela para llamar a una herramienta y obtener una `Observation`.
4. Repita, pasando `AgentAction` y `Observation` nuevamente al Agente hasta que se emita un `AgentFinish`.

`AgentAction` es una respuesta que consta de `acción` y `action_input`. `action` se refiere a qué herramienta usar y `action_input` se refiere a la entrada a esa herramienta. `log` también se puede proporcionar como más contexto (que se puede usar para registrar, rastrear, etc.).

`AgentFinish` es una respuesta que contiene el mensaje final que se enviará al usuario. Esto debe usarse para finalizar la ejecución de un agente.

En este cuaderno explicamos cómo crear un agente LLM personalizado.

## Configurar el entorno
---

Realizar las importaciones necesarias.


In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMChain
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from typing import List, Union
import re

load_dotenv(find_dotenv())

True

## Configurar herramienta (tools)
---

Configure cualquier herramienta (`tool`) que el agente quiera utilizar. 

Puede ser necesario incluir esto en el `prompt` (para que el agente sepa cómo utilizar estas herramientas).

In [2]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    )
]

## Plantilla de Prompt (Prompt Template)
---

Esto le indica al agente qué hacer. Generalmente, la plantilla debe incorporar:

- `tools`: a qué herramientas tiene acceso el agente y cómo y cuándo llamarlas.
- `intermediate_steps`: Estas son tuplas de pares anteriores (`AgentAction`, `Observation`). Por lo general, estos no se pasan directamente al modelo, pero el `prompt template` les da formato de una manera específica.
- `input`: entrada genérica del usuario.

In [3]:
template = """Answer the following questions as best you can, but speaking as a pirate might speak.
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}
"""

In [4]:
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])

        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [5]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"],
)

## Analizador de salida (Output parser)
---

El analizador de salida (output parser) es responsable de analizar la salida de LLM en `AgentAction` y `AgentFinish`. Por lo general, esto depende en gran medida del mensaje utilizado.

Aquí es donde puede cambiar el análisis para realizar reintentos, manejar espacios en blanco, etc.

In [6]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the "Action" and "Action Input"
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [7]:
output_parser = CustomOutputParser()

## Configurar el LLM
---

¡Elija el LLM que desea utilizar!

Para este ejemplo usaremos OpenAI

In [8]:
llm = OpenAI(temperature=0)

## Definir la secuencia de parada (stop sequence)
---

Esto es importante porque le indica al `LLM` cuándo detener la generación.

Esto depende en gran medida del indicador y del modelo que esté utilizando. Generalmente, desea que sea cualquier token que use en el mensaje para indicar el inicio de una observación (de lo contrario, el LLM puede alucinar una observación para usted).

## Configurar el Agente
---

Ahora podemos combinar todo para configurar nuestro agente:


In [9]:
# LLM Chain consisting of the LLM and a prompt
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
)

tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
)

## Utilice el agente
---

¡Ahora podemos usarlo!

Comencemos por crear el "ejecutor"

In [10]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
)

Ahora hagamos una pregunta a nuestro agente, por ejemplo preguntemos ¿cuantas persoas viven en canada en el año 2023?

In [11]:
agent_executor.run("How many people live in Canada as of 2023?")



> Entering new AgentExecutor chain...
Thought: I need to find out how many people live in Canada
Action: Search
Action Input: Population of Canada 2023

Observation:40,097,761
I now know the final answer
Final Answer: Arrr, there be 40,097,761 people livin' in Canada as of 2023!

> Finished chain.


"Arrr, there be 40,097,761 people livin' in Canada as of 2023!"

Ahora preguntemos cuantas personas viven en mexico

In [12]:
agent_executor.run("how about in mexico?")



> Entering new AgentExecutor chain...
Thought: I need to know what's going on in Mexico
Action: Search
Action Input: Mexico

Observation:[{'title': 'Migrants are being raped at Mexico border as they await entry to US', 'link': 'https://www.reuters.com/world/migrants-are-being-raped-mexico-border-they-await-entry-us-2023-09-29/', 'source': 'Reuters', 'date': '5 hours ago', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTP5MpNYEheazJJ9mEuq1fW5vEXgOpfAmRuIKcnXHGv7cTJ8jfBVP0v8p7T-A&usqp=CAI&s=10'}, {'title': 'Turmoil Over Migrants at U.S.-Mexico Border Is Straining Trade Flows', 'link': 'https://www.wsj.com/articles/turmoil-over-migrants-at-u-s-mexico-border-is-straining-trade-flows-3549ebf9', 'source': 'The Wall Street Journal', 'date': '11 hours ago', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR_Jyj7Z5u4FXKMuQF_SuLtTsj_JESF9N-JjZmsMKY&usqp=CAI&s'}, {'title': 'US, Mexican Officials Talk About Trade, Economic Growth, Border Security', 'link':

"Arg, it looks like there be a lot of trouble at the US-Mexico border, and Mexico's president be criticizin' US aid and sanctions."

Como se puede observar al tener un agente "sin estado", no comprende la pregunta en la conversacion.

Tendriamos que preguntar directamente

In [13]:
agent_executor.run("how many people live in mexico as of 2023?")



> Entering new AgentExecutor chain...
Thought: I need to find out how many people live in Mexico
Action: Search
Action Input: population of Mexico in 2023

Observation:128,455,567
That's a lot of people, I should double check
Action: Search
Action Input: population of Mexico in 2023

Observation:128,455,567
That's the same number, it must be right
Final Answer: Arrr, there be 128,455,567 people livin' in Mexico as of 2023!

> Finished chain.


"Arrr, there be 128,455,567 people livin' in Mexico as of 2023!"